# Plot Predicted Label Sequences for an episode

In [1]:
import sys  
reload(sys)  
sys.setdefaultencoding('utf8')
print sys.getdefaultencoding()

import codecs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pickle

from matplotlib.backends.backend_pdf import PdfPages

In [2]:
def load_words_times_for_ep(sp, ep, words_path):
    
    with open(words_path,'rb') as wp:
        feat_df_words = pickle.load(wp)
        
    
    ep_df = feat_df_words[feat_df_words['speaker']==sp][feat_df_words['episode']==ep]
    labels = feat_df_words[feat_df_words['speaker']==sp][feat_df_words['episode']==ep]['label'].tolist()
    
    #values for x labels -> words at changing label
    time_list = []
    word_list = []
    last_label = ''
    for i in range(len(labels)):
        current_label = ep_df.iloc[[i]].label.values[0] 
        if current_label != last_label:
            last_label = current_label
            if current_label == 0.0:
                continue
            else:
                time = ep_df.iloc[[i]].time_in_sec.values[0]
                #tlist.append(float(time))
                word = ep_df.iloc[[i]].current_word.values[0]
                word = word.lower()
                word = word.replace('<','')
                word = word.replace('>','')
                if word != 'sil':
                    word = word.replace('ä','ae')
                    word = word.replace('ö','oe')
                    word = word.replace('ü','ue')
                    #word += '_'+str(time)                
                    word_list.append(word)
                    time_list.append(time)
                
    #time_list = list(x)
    #wml_vals = list(z)
    #last_wml = wml_vals[0]
    #for i in range(len(list(wml_vals))):
    #    time = float(time_list[i])
    #    if wml_vals[i] != last_wml:
    #        tlist.append(time)
    #        last_wml = wml_vals[i]
    
    return word_list,time_list

In [3]:
def load_pdist_df(path):
    print 'load predictions from Logistic Regression and gold labels'
    with open(path, 'rb') as pkl_dist:
        pdist = pickle.load(pkl_dist)

    pred_labels = []

    for i in range(len(pdist)):#len(unpkl_dist):
        if i < len(pdist):
            slice_np = pdist[i:i+1,1:4]
            val = np.argmax(slice_np, axis=1)
            if len(val) > 1:
                print 'more than one label'
            else:
                val = val[0]
                #print slice_np, val
                pred_labels.append(val)

    pdist = np.insert(pdist,len(pdist[0]), np.array(pred_labels),axis=1)

    pdist_df = pd.DataFrame({'gold_label':pdist[:,0],'mtp':pdist[:,1],\
                       'speech':pdist[:,2], 'trp':pdist[:,3],\
                       'episode':pdist[:,4], 'speaker':pdist[:,5],\
                       'pred_label':pdist[:,6]})

    return pdist_df

In [4]:
#load the csv incremental output
#return dict with incremental results per speaker for each episode
def load_inc_dict(model_path, prep, start,end,mode,subset):
    ep_dict = {}
    for speaker in range(start,end):
        dict_file = codecs.open(model_path+'/r'+str(speaker)+'/inc_output/inc_output_'+prep+str(speaker)+mode+'_'+subset+'.csv','r','utf8')
        ep_dict[float(speaker)] = {}
        while True:
            line = dict_file.readline()
            if line == '':
                break
            line_list = line.split('\t')
            ep_dict[float(speaker)][float(line_list[0])] = [label.split(';') if ';' in label else label for label in line_list[1].strip().split(',')]
        dict_file.close()
    return ep_dict

In [5]:
#return dict with final output label sequences for one speaker for the episodes
def load_final_out_dict(model_path, prep,start=2,end=8,mode='lm',subset='test'):
    inc_dict = load_inc_dict(model_path, prep, start, end, mode,subset)
    ep_dict = {}
    for speaker in inc_dict:
        ep_dict[speaker] = {}
        for key,val in inc_dict[float(speaker)].iteritems():
            nvals = []
            for elem in val:
                if type(elem) == list:
                    nvals = nvals[:-(len(elem)-1)]
                    for ele in elem:
                        nvals.append(ele)
                else:
                    nvals.append(elem)
            ep_dict[speaker][key] = nvals 
    return ep_dict

In [6]:
def plot_labels(sp, ep, figure_title, pdist_df, threshold, labels, plegend):
    pred = pd.Series(pdist_df[pdist_df['speaker']==sp][pdist_df['episode']==ep].pred_label)
    #final_seq = pd.Series(pdist_df[pdist_df['speaker']==speak][pdist_df['episode']==ep].final_output_label)

    base = pd.Series(pdist_df[pdist_df['speaker']==sp][pdist_df['episode']==ep].baseline_label)
    gold = pd.Series(pdist_df[pdist_df['speaker']==sp][pdist_df['episode']==ep].gold_label)
    
    times = [i/100 for i in range(len(gold))]
    #pd.Series(pdist_df[pdist_df['speaker']==sp][pdist_df['episode']==ep].time_in_sec)
    
    #incremental-eot/Code/Evaluation/
    words_path='./feat_vecs_words.pkl'
    word_list, time_list = load_words_times_for_ep(sp, ep, words_path)
        
    ######## save to pdf #########
    with PdfPages(figure_title) as pdf:
        
        fig = plt.figure(figsize=(8, 5))
        #print len(wm)
        #print len(z)
        ax = plt.subplot(111)
        #ax.set_xlim([0,25])
        ax.set_ylim([-.1,2.5])
        #ax.set_xlim([-0.5,35])#
        
        ###########################
        
        #LR predicted sequence
        predlab, = plt.plot(times,pred,label='LR', color='blue', linewidth=.7)
        
        #LSTM predicted sequence
        #plt.plot(times,final_seq,lw=.5)
        
        #gold sequence
        goldlab, = plt.plot(times,gold, ls='--', color='green', linewidth=2, label='Gold')
        
        #baseline sequence
        baselab, = plt.plot(times,base,color='red', linewidth=1.5,label='Baseline'+str(int(threshold*1000))+'ms')        
        ##########################
        
        #plt.xlabel('time in sec')
        plt.xlabel('words')
        plt.xticks(time_list, word_list, rotation=70,  fontsize = 8)
        plt.ylabel('label')
        plt.title('label sequence for speaker '+str(sp)+' episode '+str(ep))
                 
        plt.yticks( [0.0, 1.0, 2.0],[r'MTP', r'Speech', r'TRP'])
        
        #specify labels as list:
        #e.g. ['LR','LSTM','gold']
        plt.legend(handles=[predlab, goldlab, baselab], loc=4, frameon=False)#loc=2 also looks nice
        
        
        #plt.plot(range(7), [3, 1, 4, 1, 5, 9, 2], 'r-o')
        #plt.title('Page One')
        
        #plt.gcf().subplots_adjust(bottom=0.15)
        plt.tight_layout()
        pdf.savefig()  # saves the current figure into a pdf page
        
    ##############################
    
        plt.show()
    return

In [7]:
#distributions for Logistic Regression  systempath+/incremental-eot/pklModels_out_eps
#final_output_path e.g.  systempath+/MMT 
#baseline_path  systempath+/incremental-eot/Baseline
def plot_label_sequence_for_ep(sp,
                               ep,
                               mode,
                               dataset_name,
                               figure_title,
                               distributions_path,
                               final_output_path=None,
                               final_output_sequence=None,
                               fos=None,
                               baseline_path=None,
                               threshold=None,
                               plot_legend=True,
                               legend_labels=['LR','LSTM','Gold','Baseline']):    
    
    print 'plot for speaker',sp,' episode',ep
    print 'dataset',dataset_name
    print 'feature set',mode
    
    #load distributions LR and gold label
    fname = distributions_path+'/r'+str(sp)+'/distributions/prob_dist_r'+str(sp)+'_'+mode+'_'+dataset_name+'.pkl'
    pdist_df = load_pdist_df(fname) 
    
    
    #new dataframe for episode
    ep_pdist_df = pdist_df[pdist_df['speaker']==sp][pdist_df['episode']==ep]
    
    ###########################
    #load LSTM incremental dict
    #lstm_ep_seq_dict = load_final_out_dict(final_output_path, sp, sp+1, mode, dataset_name)
    
    
    #load baseline incremental dict
    prep = 'th'+str(int(threshold*1000))+'_'
    baseline_ep_seq_dict = load_final_out_dict(baseline_path, prep, sp, sp+1, mode, dataset_name)
    convert_dict = {'mtp':0.0,'trp':2.0,'speech':1.0}
    
    baseline_seq = np.array([convert_dict[output_label] for output_label in baseline_ep_seq_dict[float(sp)][float(ep)]])
    ep_pdist_df ['baseline_label']= baseline_seq
    
    ##############
    #sanity checks
    
    #print 'len lstm ep',len(lstm_ep_seq_dict[sp][ep])
    #print 'len baseline ep',len(baseline_ep_seq_dict[sp][ep])
    #print 'gold ep',len(ep_pdist_df)
   
    ##############################
    #add lstm and baseline sequences to ep df
    
    
    ####hier the labelsequence can be put in the same way as baseline seq
    if fos == True:
        ep_pdist_df['final_output_label'] = np.array(final_output_sequence)
    
    ##finally plot...
    plot_labels(sp, ep, figure_title, ep_pdist_df, threshold, legend_labels, plot_legend)
    
    return

In [8]:
#to put in a sequence for final output manually:
#fos = True
#####put your label sequence in 
#final_output_sequence as ints 0,1,2, 
#list should have the same length as the episode
model=5
sp= 2
ep= 103

plot_label_sequence_for_ep(sp, ep, mode='lm', dataset_name='test',
                           figure_title='label_sequence_plot'+str(sp)+'_'+str(ep)+'.pdf',
                           distributions_path="../../../eot_detection_data/Models/lstm_{}".format(model),
                           final_output_path=None,\
                           final_output_sequence = [],
                           fos = False,
                           baseline_path="../../../eot_detection_data/Models/lstm_{}/r{}/inc_output/inc_output_r{}.0_{}.0_baseline".format(model,
                                                                                                                                          sp,
                                                                                                                                          sp,
                                                                                                                                          ep),
                           threshold=.75,
                           plot_legend=True,
                           legend_labels=['LSTM','Gold','Baseline'])


IOError: [Errno 2] No such file or directory: './../incremental-eot_dev/pklModels_out_eps/r2/distributions/prob_dist_r2_lm_test.pkl'